In [1]:
import pandas as pd
import PyPDF2
import numpy as np
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 100)

In [5]:
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')
pdf_file = open('data/00-391.pdf',"rb")
read_pdf = PyPDF2.PdfFileReader(pdf_file)

number_of_pages = read_pdf.getNumPages()
page = read_pdf.getPage(0)
page_content = page.extractText()
print(page_content)

                       1             IN THE SUPREME COURT OF THE UNITED STATES
            2    - - - - - - - - - - - - - - - -X
            3    FLORIDA,                       :
            4              Petitioner           :
            5         v.                        :  No. 00-391
            6    ROBERT A. THOMAS               :
            7    - - - - - - - - - - - - - - - -X
            8                                 Washington, D.C.
            9                                 Wednesday, April 25, 2001
           10              The above-entitled matter came on for oral
           11    argument before the Supreme Court of the United States at
           12    11:14 a.m.
           13    APPEARANCES:
           14    ROBERT J. KRAUSS, ESQ., Senior Assistant Attorney
           15         General, Tampa, Florida; on behalf of the Petitioner.
           16    GREGORY G. GARRE, ESQ., Assistant to the Solicitor
           17         General, Department of Justice, Washin

In [3]:
type(page_content)

str

In [4]:
len(page_content)

1390

In [6]:
dk =[]
number_of_pages = read_pdf.getNumPages()
for i in range(0,number_of_pages):
    page = read_pdf.getPage(i)
    page_content = page.extractText()
    dk.append(page_content)

In [9]:
#Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')
#pd.set_option('display.max_columns', 100)

In [55]:
#Cases[Cases['dateArgument']=='3/26/2012']

### Need to use docket from filename xxxx.pdf to tie back case to other data

In [56]:
#Cases[Cases['docket']=='00-1021']

In [12]:
##download each PDF.
## Access its text
##put text in dataframe with docket# and map to original Cases data


In [13]:
files = os.listdir('ArgText/')
paths = []
for file in files:
    paths.append('ArgText/'+file)
len(paths)

1311

#### import pdf filename and contents into dictionary

In [14]:
my_dict = {}
for index, path in enumerate(paths):
    with open(path, "rb") as f:
        dk = []
        read_pdf = PyPDF2.PdfFileReader(f)
        for i in range(0,read_pdf.getNumPages()):
            page = read_pdf.getPage(i)
            page_content = page.extractText()
            dk.append(page_content)
    my_dict[files[index].replace(".pdf", "")] = " ".join(dk)##removes pdf from key and turns value array into string

In [ ]:
#files = os.listdir('ArgText/')
#paths = []
#for file in files:
    #paths.append('ArgText/'+file)

In [ ]:
#pdf_file = open('data/00-391.pdf',"rb")
#read_pdf = PyPDF2.PdfFileReader(pdf_file)

#number_of_pages = read_pdf.getNumPages()
#page = read_pdf.getPage(36)
#page_content = page.extractText()

In [57]:
len(my_dict)

1311

In [67]:
my_dict['01-7574']

'1234\n5\n678\n910\n11\n12\n131415\n161718\n19\n202122\n232425 IN THE SUPREME COURT OF THE UNITED STATES•- - - - - - - - - - - - - - - -X•DAVID ALLEN SATTAZAHN, :•Petitioner :•v. : No. 01-7574•PENNSYLVANIA :•- - - - - - - - - - - - - - - -X•Washington, D.C.•Monday, November 4, 2002•The above-entitled matter came on for oral•argument before the Supreme Court of the United States at•11:06 a.m.•\nAPPEARANCES:•\nROBERT B. DUNHAM, ESQ., Assistant Federal Defender,•Philadelphia, Pennsylvania; on behalf of the Petitioner.•IVA C. DOUGHERTY, ESQ., First Assistant District Attorney,•Reading, Pennsylvania; on behalf of the Respondent.•SRI SRINIVASAN, ESQ., Assistant to the Solicitor General,•Department of Justice, Washington, D.C.; on behalf of•the United States, as amicus curiae, supporting the•Respondent.•1  1234\n5\n678\n910\n11\n12\n131415\n161718\n19\n202122\n232425 C O N T E N T S•ORAL ARGUMENT OF •ROBERT B. DUNHAM, ESQ.•On behalf of the Petitioner •ORAL ARGUMENT OF•\nIVA C. DOUGHERTY, ESQ.

### Convert dictionary to dataframe and attach labels

In [17]:
Labels = Cases[['docket','partyWinning']]

In [18]:
Text = pd.DataFrame.from_dict(my_dict, orient="index")

In [19]:
Text = Text.rename(index=str, columns={0: "Document"})

In [59]:
def GetMaxItem(item):        
    maks=max(item, key=lambda k: len(item[k]))
    return len(item[maks]), maks
GetMaxItem(my_dict)

(265483, '02-1674')

#### Checking and reindexing new dataframe

In [33]:
Text.head()

,Document,docket,partyWinning
0,1 IN THE SU...,00-1011,1.0
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021,0.0
2,1 2 \n ...,00-1045,1.0
3,12345678910\n111213141516171819202122232425 IN...,00-10666,0.0
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072,1.0


In [25]:
Text['docket'] = Text.index

In [26]:
Text.reset_index(drop=True)

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072
5,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1073
6,1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...,00-1089
7,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1167
8,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1187
9,1 IN THE SU...,00-121


#### attaching labels using inner join

In [27]:
Text = pd.merge(Text, Labels, on='docket', how='inner')

##### look at length of docs/sentences/wordcounts

In [28]:
num_words = Text['Document'].apply(lambda x: len(x.split()))
num_words_mean, num_words_std = np.mean(num_words), np.std(num_words)

num_sentences = Text['Document'].apply(lambda x: len(re.split( '~ ...' ,'~'.join(x.split('.')))))
num_sentences_mean = np.mean(num_sentences)

In [54]:
Text.head()

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072


In [39]:
Labels = Text[['partyWinning']]

In [41]:
Text.drop('partyWinning', axis=1, inplace= True)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(Text.Document, Labels, test_size=0.2, random_state=23)

In [87]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), stop_words='english', max_df=.50)),               
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),])
text_clf = text_clf.fit(X_train, y_train)

In [73]:
y_test = np.ravel(y_test)
y_train = np.ravel(y_train)

In [80]:
text_clf

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [81]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.6833976833976834

In [136]:
y_test

array([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 0.

In [101]:
len(y_test)

259

In [137]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(ngram_range=(1,3), stop_words='english', max_df=.30)),
                     ('tfidf', TfidfTransformer()),
                 ('clf-svm', SGDClassifier(loss='log', penalty='l2',
                                          alpha=.0001, random_state=23, n_jobs=-1,class_weight= {0:.32, 1: 0.68})),])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.6756756756756757

In [141]:
predicted_svm

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [140]:
X_test

85      1234\n567\n8910111213\n141516\n171819\n202122\...
952        ˙\n$& ' #-        \n \n   \n\n\n\n\n\n\n\n ...
892        ˇ \n˚\n') & *   \n           \n\n\n\n\n\n\n...
545                                                   ...
732                                                   ...
106     1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...
651                                                   ...
836                                                   ...
211     12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...
658                                                   ...
586                                                   ...
1054    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
678                                                   ...
350                                               1234...
989     \n\n\n\n   \n˙\n $& ) #.       \n \n \n \n \n ...
1059    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
454                                                   ...
756           

In [130]:
177/259

0.6833976833976834

In [126]:
y_test

array([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 0.

In [142]:
def preprocess(data):
    reviews_tokens = []
    for review in data:
        review = review.lower() #Convert to lower-case words
        raw_word_tokens = re.findall(r'(?:\w+)', review,flags = re.UNICODE) #remove pontuaction
        word_tokens = [w for w in raw_word_tokens if not w in stop_words] # do not add stop words
        reviews_tokens.append(word_tokens)
    return reviews_tokens #return all tokens

In [143]:
stop_words =['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself',
            'yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself',
            'they','them','their','theirs','themselves','what','which','who','whom','this','that',
            'these','those','am','is','are','was','were','be','been','being','have','has','had',
            'having','do','does','did','doing','a','an','the','and','but','if','or','because','as',
            'until','while','of','at','by','for','with','about','against','between','into','through',
            'during','before','after','above','below','to','from','up','down','in','out','on','off',
            'over','under','again','further','then','once','here','there','when','where','why','how',
            'all','any','both','each','few','more','most','other','some','such','no','nor','not',
            'only','own','same','so','than','too','very','s','t','can','will','just','don','should',
            'now','uses','use','using','used','one','also']

In [144]:
def make_word_bag(data):
    corpus = preprocess(data)
    bag_of_words = {}
    word_count = 0
    for sentence in corpus:
        for word in sentence:
            if word not in bag_of_words: 
                bag_of_words[word] = word_count #set indexes
                word_count+=1
    return bag_of_words #index of letters

bag_of_words = make_word_bag(Text.Document)

In [145]:
def features(sentence_tokens,bag_of_words):
    sentence_features = [0 for x in range(len(bag_of_words))]
 
    for word in sentence_tokens:
        index = bag_of_words[word]
        sentence_features[index] +=1
    return sentence_features

In [146]:
def get_rev_features(data,bag_of_words):
    review_features = []
    reviews_text_tokens = preprocess(data)
    for review_text in reviews_text_tokens:
        feature_review_text = features(review_text,bag_of_words)
        review_features.append(feature_review_text)
    return review_features
 
rev_features = get_rev_features(Text.Document,bag_of_words)

In [147]:
X_train, X_test, y_train, y_test = train_test_split( rev_features, Labels, test_size=0.2, random_state=23)

In [154]:
clf2 = MultinomialNB(alpha=.01)
 
clf2.fit(X_train, y_train)
 
pred = clf2.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.587


In [153]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics